In [2]:
from sklearn.svm import SVC
 
import argparse
 
import mahotas
 
import glob
 
import cv2
 
import numpy as np
import pandas as pd

In [7]:
def describe(image):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
    hara = mahotas.features.haralick(image).mean(axis=0)
 
    return hara
def track_hara(text):
    haralick_name = ["id",
              "Angular Second Moment",
              "Contrast",
              "Correlation",
              "Variance",
              "Inverse Difference Moment",
              "Sum Average",
              "Sum Variance",
              "Sum Entropy",
              "Entropy ",
              "Difference Variance",
              "Difference Entropy",
              "f12",
              "f13",
              "label"]

    table = pd.DataFrame(columns=haralick_name)
    print("total img n:",len(text))
    for i in range(len(text)):
        l = text[i][0].rfind("/")+1
        r = text[i][0].rfind(".")
        name = text[i][0][l:r]
        lable = int(text[i][1])
        image = cv2.imread("./"+text[i][0])
        feature  = describe(image).tolist()
        temp_list = [[name]+feature+[lable]]
        temp_table = pd.DataFrame(temp_list,columns=haralick_name)
        table = table.append(temp_table, ignore_index=True)
        if (i % 2000)==0:
            print("finish:",i)
    return(table)
def top_n_acc(pred_table,truth,n):
    acc = []
    f_n = len(pred_table.iloc[0])
    q = (1-(n/f_n))*100
    for i in range(len(pred_table)):
        target = pred_table.loc[i,pred_table.iloc[i,:]>np.percentile(pred_table.iloc[i,:],q)].index
        if testy.iloc[i]["label"] in target:
            acc.append(1)
        else:
            acc.append(0)
    output = sum(acc)/len(acc)
    return output

In [13]:
trainpath = 'train.txt'

f = open(trainpath,"r")
text = []
for line in f:
    temp = line.split(' ')
    temp[1] = temp[1][:-1]
    text.append(temp)
f.close

table = track_hara(text)
table.to_csv("train_haralick.csv", index=False)

total img n: 63325
finish: 0
finish: 2000
finish: 4000
finish: 6000
finish: 8000
finish: 10000
finish: 12000
finish: 14000
finish: 16000
finish: 18000
finish: 20000
finish: 22000
finish: 24000
finish: 26000
finish: 28000
finish: 30000
finish: 32000
finish: 34000
finish: 36000
finish: 38000
finish: 40000
finish: 42000
finish: 44000
finish: 46000
finish: 48000
finish: 50000
finish: 52000
finish: 54000
finish: 56000
finish: 58000
finish: 60000
finish: 62000


In [111]:
testpath = 'test.txt'

f = open(testpath,"r")
text = []
for line in f:
    temp = line.split(' ')
    temp[1] = temp[1][:-1]
    text.append(temp)
f.close

table = track_hara(text)
table.to_csv("test_haralick.csv", index=False)

total img n: 450
finish: 0


In [113]:
valpath = 'val.txt'

f = open(valpath,"r")
text = []
for line in f:
    temp = line.split(' ')
    temp[1] = temp[1][:-1]
    text.append(temp)
f.close

table = track_hara(text)
table.to_csv("val_haralick.csv", index=False)

total img n: 450
finish: 0


In [3]:
train_table = pd.read_csv("train_haralick.csv")
test_table = pd.read_csv("test_haralick.csv")
val_table = pd.read_csv("val_haralick.csv")

In [4]:
trainX = train_table.iloc[:,1:14]
trainy = train_table.iloc[:,14:]

testX = test_table.iloc[:,1:14]
testy = test_table.iloc[:,14:]

valX = val_table.iloc[:,1:14]
valy = val_table.iloc[:,14:]

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [129]:
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(trainX, trainy)

test_prob = pd.DataFrame(clf.predict_proba(testX))
val_prob = pd.DataFrame(clf.predict_proba(valX))

test_acc_1 = top_n_acc(test_prob,testy,1)
test_acc_5 = top_n_acc(test_prob,testy,5)
val_acc_1 = top_n_acc(val_prob,testy,1)
val_acc_5 = top_n_acc(val_prob,testy,5)
print("test_acc_1",test_acc_1)
print("test_acc_5",test_acc_5)
print("val_acc_1",val_acc_1)
print("val_acc_5",val_acc_5)

<ipython-input-129-c8d9c6a5e169>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(trainX, trainy)


test_acc_1 0.09111111111111111
test_acc_5 0.33555555555555555
val_acc_1 0.08666666666666667
val_acc_5 0.3088888888888889


In [11]:
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(loss_function='MultiClass',
                           eval_metric="AUC",
                           learning_rate=0.01,
                           iterations=1000,
                           random_seed=42,
                           od_type="Iter",
                           depth=10,
                           early_stopping_rounds=500
                          )
cat_model.fit(trainX,trainy, eval_set=(testX, testy), verbose=0, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [12]:
cat_test_prob = pd.DataFrame(cat_model.predict_proba(testX))
cat_val_prob = pd.DataFrame(cat_model.predict_proba(valX))
cat_test_acc_1 = top_n_acc(cat_test_prob,testy,1)
cat_test_acc_5 = top_n_acc(cat_test_prob,testy,5)
cat_val_acc_1 = top_n_acc(cat_val_prob,testy,1)
cat_val_acc_5 = top_n_acc(cat_val_prob,testy,5)
print("cat_test_acc_1",cat_test_acc_1)
print("cat_test_acc_5",cat_test_acc_5)
print("cat_val_acc_1",cat_val_acc_1)
print("cat_val_acc_5",cat_val_acc_5)

cat_test_acc_1 0.10222222222222223
cat_test_acc_5 0.3488888888888889
cat_val_acc_1 0.08222222222222222
cat_val_acc_5 0.32


In [141]:
from sklearn.svm import SVC
model = SVC(probability=True)


In [154]:
model.fit(trainX,trainy)

C:\Users\88697\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(probability=True)

In [155]:
SVC_test_prob = pd.DataFrame(model.predict_proba(testX))
SVC_val_prob = pd.DataFrame(model.predict_proba(valX))
SVC_test_acc_1 = top_n_acc(SVC_test_prob,testy,1)
SVC_test_acc_5 = top_n_acc(SVC_test_prob,testy,5)
SVC_val_acc_1 = top_n_acc(SVC_val_prob,testy,1)
SVC_val_acc_5 = top_n_acc(SVC_val_prob,testy,5)
print("SVC_test_acc_1",SVC_test_acc_1)
print("SVC_test_acc_5",SVC_test_acc_5)
print("SVC_val_acc_1",SVC_val_acc_1)
print("SVC_val_acc_5",SVC_val_acc_5)

SVC_test_acc_1 0.05555555555555555
SVC_test_acc_5 0.20666666666666667
SVC_val_acc_1 0.04666666666666667
SVC_val_acc_5 0.18222222222222223
